Sample code for SVD interpolation of partially unknown matrices

In [1]:
import numpy as np # library for numerics
import scipy.linalg as la # library for linear algebra
import scipy.sparse.linalg as sla # library for sparse linear algebra
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix
import random as rd # library for random numbers
import matplotlib.pyplot as plt # library for plot

For a given rating matrix $V\in \mathbb{R}^{n\times m}$, we will infer a low rank SVD ansatz $\widetilde{V}\simeq U^{T}M$, where $U\in \mathbb{R}^{f\times n}$ and $M\in \mathbb{R}^{f\times m}$.

In [2]:
class SVDansatz:

 # constructor
  def __init__(self,n,m,f):
    self.n = n # 1st linear dimension of a given matrix V
    self.m = m # 2nd linear dimension of a given matrix V
    self.f = f # assumed rank of a given matrix V
    self.V = np.zeros((n,m),dtype=np.float64) # a given partially unknown matrix V
    self.I = np.zeros((n,m),dtype=np.float64) # indicator matrix for V
    self.U = np.zeros((f,n),dtype=np.float64) # inferred left SVD unitary matrix of V 
    self.M = np.zeros((f,m),dtype=np.float64) # inferred right SVD unitary matrix of V
    self.UM = np.zeros((n,m),dtype=np.float64) # inferred rank-f SVD ansatz of V
    self.tildeV = np.zeros((n,m),dtype=np.float64) # inferred rank-f SVD ansatz of V
 
 # generate U and M
  def generateUM(self):
    self.U = np.random.random((self.f,self.n))
    self.M = np.random.random((self.f,self.m))

 # rating matrix p(U,M)
  def pUM(self):
    for i in range(self.n):
      for j in range(self.m):
        tmp = 0.0
        for k in range(self.f):
          tmp += self.U[k][i]*self.M[k][j]
        self.UM[i][j] = tmp
        if tmp <= 0.0:
          self.tildeV[i][j] = 1.0
        elif 0.0 < tmp < 4.0:
          self.tildeV[i][j] = 1.0 + tmp
        elif 4.0 <= tmp:
          self.tildeV[i][j] = 5.0

# cost function
  def cost(self,ku,km):
    costf = 0.0
    for i in range(self.n):
      for j in range(self.m):
        costf += 0.5*self.I[i][j]*((self.V[i][j]-self.tildeV[i][j])**2)
    for k in range(self.f):
      for i in range(self.n):
        costf += 0.5*ku*(self.U[k][i]**2)
      for j in range(self.m):
        costf += 0.5*km*(self.M[k][j]**2)
    return costf

# RMSE
  def RMSE(self):
    sumSE = 0.0
    sumI = 0.0
    for i in range(self.n):
      for j in range(self.m):
        sumSE += abs(self.I[i][j])*abs((self.V[i][j]-self.tildeV[i][j])**2)
        sumI   += abs(self.I[i][j])
    return np.sqrt(sumSE/sumI)

# update U and M (nabla of pUM is replaced by nabla of 1 + U^T M)
# pUM <= 1 + U^T M
  def update_UM(self,ku,km,mu):
    nablaU_E = np.zeros((self.f,self.n),dtype=np.float64)
    nablaM_E = np.zeros((self.f,self.m),dtype=np.float64)
    VMU = np.zeros((self.n,self.m),dtype=np.float64)
    for k in range(self.n):
      for j in range(self.m):
        VMU[k][j] = self.I[k][j]*(self.V[k][j] - 1.0 - self.UM[k][j])
    nablaU_E = (-1.0)*np.matmul(self.M,VMU.T)
    nablaM_E = (-1.0)*np.matmul(self.U,VMU)
    nablaU_E[:][:] += ku*self.U[:][:]
    nablaM_E[:][:] += km*self.M[:][:]
    self.U[:][:] -= mu*nablaU_E[:][:]
    self.M[:][:] -= mu*nablaM_E[:][:]
    self.pUM()

# generate I
  def generateI(self):
    self.I[:][:] = 0.0
    for i in range(self.n):
      for j in range(self.m):
        if self.V[i][j] != 0.0:
          self.I[i][j] = 1.0

# Gram-Schmidt
  def GSD(self):
    tmpU = np.zeros((self.f,self.n),dtype=np.float64)
    for i in range(self.f):
      tmpU[i][:] = self.U[i][:]
      for k in range(i):
        uv = 0.0
        uu = 0.0
        for j in range(self.n):
          uv += tmpU[k][j]*self.U[i][j]
          uu += tmpU[k][j]**2
        tmpU[i][:] -= (uv/uu)*tmpU[k][:]
    for i in range(self.f):
      norm = 0.0
      for j in range(self.n):
        norm += tmpU[i][j]**2
      tmpU[i][:] = tmpU[i][:]/np.sqrt(norm)
    self.U[:][:] = (3.0*np.sqrt(1.0*self.n)/np.sqrt(1.0*self.f))*tmpU[:][:]
    tmpM = np.zeros((self.f,self.m),dtype=np.float64)
    for i in range(self.f):
      tmpM[i][:] = self.M[i][:]
      for k in range(i):
        uv = 0.0
        uu = 0.0
        for j in range(self.m):
          uv += tmpM[k][j]*self.M[i][j]
          uu += tmpM[k][j]**2
        tmpM[i][:] -= (uv/uu)*tmpM[k][:]
    for i in range(self.f):
      norm = 0.0
      for j in range(self.m):
        norm += tmpM[i][j]**2
      tmpM[i][:] = tmpM[i][:]/np.sqrt(norm)
    self.M[:][:] = (3.0*np.sqrt(1.0*self.m)/np.sqrt(1.0*self.f))*tmpM[:][:]

Below, the rank of $\widetilde{V}$ and dimension $n$ and $m$ are chosen.

In [30]:
fin = 3# guess for the rank of V
L = 20 # linear dimensions n=m of V: n=m=L 

Below, two partial matrices $A$ and $B$ are given.
Based on SVD interpolation, please infer which matrix is a random matrix.

In [31]:
A = np.zeros((L,L),dtype=np.float64)
A[19][7]=3.0
A[8][8]=2.0
A[9][4]=2.0
A[1][11]=3.0
A[4][5]=2.0
A[18][15]=2.0
A[2][5]=3.0
A[12][9]=4.0
A[2][15]=2.0
A[4][6]=2.0
A[15][4]=2.0
A[13][10]=3.0
A[10][0]=3.0
A[7][8]=2.0
A[5][4]=2.0
A[0][4]=3.0
A[10][15]=2.0
A[16][11]=2.0
A[11][8]=3.0
A[9][15]=2.0
A[13][15]=3.0
A[3][13]=2.0
A[12][2]=2.0
A[7][10]=3.0
A[13][0]=3.0
A[15][6]=2.0
A[15][18]=2.0
A[9][7]=2.0
A[8][17]=2.0
A[15][16]=2.0
A[18][16]=2.0
A[12][12]=2.0
A[5][13]=3.0
A[9][8]=3.0
A[13][12]=3.0
A[17][11]=3.0
A[18][1]=3.0
A[4][18]=2.0
A[2][5]=3.0
A[14][7]=2.0
A[3][12]=2.0
A[11][11]=3.0
A[8][6]=2.0
A[10][17]=2.0
A[17][17]=3.0
A[19][3]=3.0
A[19][16]=2.0
A[2][0]=4.0
A[3][6]=2.0
A[17][15]=2.0
A[9][3]=2.0
A[0][5]=1.0
A[0][12]=3.0
A[10][0]=3.0
A[9][4]=2.0
A[16][18]=3.0
A[6][16]=2.0
A[10][3]=2.0
A[12][9]=4.0
A[11][0]=2.0
A[14][0]=3.0
A[14][4]=2.0
A[7][15]=3.0
A[12][1]=1.0
A[11][8]=3.0
A[11][13]=2.0
A[6][6]=2.0
A[18][9]=2.0
A[11][14]=2.0
A[1][18]=1.0
A[19][17]=3.0
A[0][3]=3.0
A[7][16]=2.0
A[15][4]=2.0
A[10][10]=2.0
A[16][8]=2.0
A[8][1]=1.0
A[0][10]=2.0
A[2][10]=3.0
A[6][0]=2.0
A[12][18]=3.0
A[11][1]=3.0
A[9][17]=3.0
A[9][15]=2.0
A[1][13]=1.0
A[13][7]=3.0
A[15][3]=2.0
A[19][1]=4.0
A[9][15]=2.0
A[7][5]=4.0
A[4][3]=2.0
A[1][8]=4.0
A[3][16]=2.0
A[18][3]=2.0
A[12][13]=3.0
A[1][9]=1.0
A[10][7]=2.0
A[3][5]=2.0
A[14][1]=3.0
A[4][15]=2.0
B = np.zeros((L,L),dtype=np.float64)
B[19][7]=1.0
B[8][8]=2.0
B[9][4]=4.0
B[1][11]=4.0
B[4][5]=2.0
B[18][15]=5.0
B[2][5]=4.0
B[12][9]=3.0
B[2][15]=1.0
B[4][6]=4.0
B[15][4]=1.0
B[13][10]=5.0
B[10][0]=3.0
B[7][8]=1.0
B[5][4]=1.0
B[0][4]=2.0
B[10][15]=2.0
B[16][11]=1.0
B[11][8]=4.0
B[9][15]=3.0
B[13][15]=1.0
B[3][13]=3.0
B[12][2]=1.0
B[7][10]=3.0
B[13][0]=5.0
B[15][6]=2.0
B[15][18]=3.0
B[9][7]=1.0
B[8][17]=1.0
B[15][16]=3.0
B[18][16]=4.0
B[12][12]=5.0
B[5][13]=4.0
B[9][8]=3.0
B[13][12]=4.0
B[17][11]=4.0
B[18][1]=2.0
B[4][18]=1.0
B[2][5]=4.0
B[14][7]=1.0
B[3][12]=5.0
B[11][11]=4.0
B[8][6]=3.0
B[10][17]=1.0
B[17][17]=3.0
B[19][3]=4.0
B[19][16]=5.0
B[2][0]=3.0
B[3][6]=1.0
B[17][15]=1.0
B[9][3]=2.0
B[0][5]=1.0
B[0][12]=5.0
B[10][0]=3.0
B[9][4]=4.0
B[16][18]=3.0
B[6][16]=2.0
B[10][3]=1.0
B[12][9]=3.0
B[11][0]=1.0
B[14][0]=3.0
B[14][4]=4.0
B[7][15]=3.0
B[12][1]=1.0
B[11][8]=4.0
B[11][13]=3.0
B[6][6]=1.0
B[18][9]=2.0
B[11][14]=1.0
B[1][18]=1.0
B[19][17]=5.0
B[0][3]=3.0
B[7][16]=1.0
B[15][4]=1.0
B[10][10]=1.0
B[16][8]=1.0
B[8][1]=1.0
B[0][10]=1.0
B[2][10]=2.0
B[6][0]=1.0
B[12][18]=4.0
B[11][1]=2.0
B[9][17]=5.0
B[9][15]=3.0
B[1][13]=1.0
B[13][7]=3.0
B[15][3]=1.0
B[19][1]=4.0
B[9][15]=3.0
B[7][5]=4.0
B[4][3]=2.0
B[1][8]=4.0
B[3][16]=1.0
B[18][3]=1.0
B[12][13]=4.0
B[1][9]=1.0
B[10][7]=5.0
B[3][5]=3.0
B[14][1]=1.0
B[4][15]=2.0

In [32]:
iSVD = SVDansatz(L,L,fin) 

In the following, $N-1$ matrix elements are used for training and a single matrix element is used for estimating test error. 

In [35]:
iSVD.V[:][:] = A[:][:] # input V; you may choose A or B
iSVD.generateI() # generate indicator matrix I 
#
iSVD.generateUM() # generate initial guess for U and M
iSVD.GSD() # orthonormalize U and M
oldI = np.zeros((iSVD.n,iSVD.m),dtype=np.float64)
oldI[:][:] = iSVD.I[:][:]
icount = 0
for i in range(iSVD.n):
  for j in range(iSVD.m):
    if oldI[i][j] > 0.5:
      icount += 1
arrayij = np.zeros((2,icount),dtype=np.int32)
icount = 0
for i in range(iSVD.n):
  for j in range(iSVD.m):
    if oldI[i][j] > 0.5:
      arrayij[0][icount] = i
      arrayij[1][icount] = j
      icount += 1
# leave-one-out
testi=0
iSVD.I[:][:] = oldI[:][:]
iSVD.I[arrayij[0][testi]][arrayij[1][testi]] = 0.0
iSVD.generateUM() # generate initial guess for U and M
iSVD.GSD() # orthonormalize U and M
for i in range(2000): # you may choose the upper limit for the following iteration
    iSVD.update_UM(0.001,0.001,0.02) # ku = 0.001, km = 0.001, mu = 0.02
    if i % 200 == 0:
        print(testi,iSVD.RMSE(),iSVD.V[arrayij[0][testi]][arrayij[1][testi]]-iSVD.tildeV[arrayij[0][testi]][arrayij[1][testi]])

0 2.0815936308674527 -2.0
0 0.3045985718291471 -0.20419853087303297
0 0.2666787550183284 -0.28717344827445634
0 0.21691385518901585 -0.08539567635179957
0 0.16876791754237708 0.028095217012499507
0 0.15725175046731887 -0.027545541132790063
0 0.15264446310988425 -0.11162596138162773
0 0.14996008032015587 -0.17786203273785528
0 0.1481549341597624 -0.222215568743918
0 0.14684718258981055 -0.25035454262794987
